## LCS Player Analysis

### Stats to look into for every player
- This split's VODS:
    - Their positioning tendencies
    - Thier warding patterns
    - Rewatch TL Kay's videos again to see what to get
    - Thier picks
    - Rewatch some analysis videos
    
- Overall    
    - Stats that he provided vs other players (worlds, lcs, etc.)
        - Averages for everything + graphs
        

In [1]:
import pandas as pd

### Importing data

In [25]:
spring2020 = pd.read_excel('./data/2020/2020-spring-match-data.xlsx')  # February 17

spring2019 = pd.read_excel('./data/2019/2019-spring-match-data.xlsx')
summer2019 = pd.read_excel('./data/2019/2019-summer-match-data.xlsx')
worlds2019 = pd.read_excel('./data/2019/2019-worlds-match-data.xlsx')

spring2018 = pd.read_excel('./data/2018/2018-spring-match-data.xlsx')
summer2018 = pd.read_excel('./data/2018/2018-summer-match-data.xlsx')
worlds2018 = pd.read_excel('./data/2018/2018-worlds-match-data.xlsx')

In [50]:
# Aggregate into one giant file
data = spring2020
excel_files = [spring2019, summer2019, worlds2019, spring2018, summer2018, worlds2018]
for e in excel_files:
    data = data.append(e,sort=False)

In [51]:
len(data)

62892

### Stats that I will look into for every player

### Team Liquid

#### Doublelift

In [62]:
doublelift = data[data.player == 'Doublelift']

In [61]:
doublelift.league.value_counts()

LCS      62
NALCS    55
MSI      31
WC       12
Name: league, dtype: int64